In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings

warnings.filterwarnings(
    "ignore",
    message="plotting functions contained within `_documentation_utils` are intended for nemos's documentation.",
    category=UserWarning,
)

warnings.filterwarnings(
    "ignore",
    message="Ignoring cached namespace 'core'",
    category=UserWarning,
)

warnings.filterwarnings(
    "ignore",
    message=(
        "invalid value encountered in div "
    ),
    category=RuntimeWarning,
)

:::{admonition} Download
:class: important render-all

This notebook can be downloaded as **{nb-download}`02_current_injection-users.ipynb`**. See the button at the top right to download as markdown or pdf.
:::
:::{admonition} Jupyter Lab tip
:class: important render-all

Newer versions of Jupyter Lab have addressed an issue with skipping around the notebook while scrolling. To make sure this fix is enabled, in the Jupyter Lab GUI, navigate to `Settings > Settings Editor > Notebook` and scroll down to the `Windowing mode` setting and make sure it is set to `contentVisibility`. 

Also reminder to presenter: Go to `View > Appearance`, select `Simple Interface` and turn off everything else to hide as many bars as possible. And maybe activate `Presentation Mode`.

And turn on `View > Render side-by-side` (shortcut `Shift+R`).
:::
# Introduction to GLM and NeMoS
This notebook has had all its explanatory text removed and has not been run.
 It is intended to be downloaded and run locally (or on the provided binder)
 while listening to the presenter's explanation. In order to see the fully
 rendered of this notebook, go [here](../../full/live_coding/02_current_injection.md)

Data for this notebook is a patch clamp experiment with a mouse V1 neuron, from the [Allen Brain Atlas](https://celltypes.brain-map.org/experiment/electrophysiology/478498617)



![Allen Brain Atlas view of the data we will analyze.](../../_static/allen_data.png)




## Learning objectives 

- Learn how to explore spiking data and do basic analyses using pynapple
- Learn how to structure data for NeMoS
- Learn how to fit a basic Generalized Linear Model using NeMoS
- Learn how to retrieve the parameters and predictions from a fit GLM for
  intrepetation.

In [ ]:
import workshop_utils
# Import everything
import jax
import matplotlib.pyplot as plt
import numpy as np
import pynapple as nap

import nemos as nmo

# configure pynapple to ignore conversion warning
nap.nap_config.suppress_conversion_warnings = True

# some helper plotting functions
from nemos import _documentation_utils as doc_plots

# configure plots some
plt.style.use(nmo.styles.plot_style)

## Data Streaming

- Stream the data. Format is [Neurodata Without Borders (NWB) standard](https://nwb-overview.readthedocs.io/en/latest/)

In [ ]:
path = workshop_utils.fetch_data("allen_478498617.nwb")

## Pynapple
### Data structures and preparation

- Open the NWB file with [pynapple](https://pynapple.org)

In [ ]:
data = nap.load_file(path)
print(data)

![Annotated view of the data we will analyze.](../../_static/allen_data_annotated.gif)
<!-- this gif created with the following imagemagick command: convert -layers OptimizePlus -delay 100 allen_data_annotated-units.svg allen_data_annotated-epochs.svg allen_data_annotated-stimulus.svg allen_data_annotated-response.svg -loop 0 allen_data_annotated.gif -->

- `stimulus`: injected current, in Amperes, sampled at 20k Hz.
- `response`: the neuron's intracellular voltage, sampled at 20k Hz. We will not use this info in this example.
- `units`: dictionary of neurons, holding each neuron's spike timestamps.
- `epochs`: start and end times of different intervals, defining the experimental structure, specifying when each stimulation protocol began and ended.

In [ ]:
trial_interval_set = data["epochs"]

current = data["stimulus"]
spikes = data["units"]

In [ ]:
trial_interval_set

- `Noise 1`: epochs of random noise

In [ ]:
noise_interval = trial_interval_set[trial_interval_set.tags == "Noise 1"]
noise_interval

- Let's focus on the first epoch.

In [ ]:
noise_interval = noise_interval[0]
noise_interval

- `current` : Tsd (TimeSeriesData) : time index + data

In [ ]:
current

- `restrict` : restricts a time series object to a set of time intervals delimited by an IntervalSet object

In [ ]:
current = current.restrict(noise_interval)
# convert current from Ampere to pico-amperes, to match the above visualization
# and move the values to a more reasonable range.
current = current * 1e12
current

- `TsGroup` : a dictionary-like object holding multiple `Ts` (timeseries) objects with potentially different time indices.

In [ ]:
spikes

We can index into the `TsGroup` to see the timestamps for this neuron's spikes:

In [ ]:
spikes[0]

Let's restrict to the same epoch `noise_interval`:

In [ ]:
spikes = spikes.restrict(noise_interval)
print(spikes)
spikes[0]

Let's visualize the data from this trial:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 2))
ax.plot(current, "grey")
ax.plot(spikes.to_tsd([-5]), "|", color="k", ms = 10)
ax.set_ylabel("Current (pA)")
ax.set_xlabel("Time (s)")

(current-inj-basic-users)=
### Basic analyses

The Generalized Linear Model gives a predicted firing rate. First we can use pynapple to visualize this firing rate for a single trial.

- `count` : count the number of events within `bin_size`

In [ ]:
# bin size in seconds
bin_size = 0.001
# Get spikes for neuron 0
count = spikes[0].count(bin_size)
count

Let's convert the spike counts to firing rate :

- `smooth` : convolve with a Gaussian kernel

In [ ]:
# the inputs to this function are the standard deviation of the gaussian in seconds and
# the full width of the window, in standard deviations. So std=.05 and size_factor=20
# gives a total filter size of 0.05 sec * 20 = 1 sec.
firing_rate = count.smooth(std=0.05, size_factor=20)
# convert from spikes per bin to spikes per second (Hz)
firing_rate = firing_rate / bin_size

In [ ]:
print(type(firing_rate))

In [ ]:
doc_plots.current_injection_plot(current, spikes, firing_rate);

What is the relationship between the current and the spiking activity?
[`compute_tuning_curves`](https://pynapple.org/generated/pynapple.process.tuning_curves.html#pynapple.process.tuning_curves.compute_tuning_curves) : compute the firing rate as a function of some feature(s).

In [ ]:
tuning_curve = nap.compute_tuning_curves(spikes, current, bins=15, feature_names=["current"])
tuning_curve

Let's plot the tuning curve of the neuron.

In [ ]:
doc_plots.tuning_curve_plot(tuning_curve);

## NeMoS 
(current-inj-prep-users)=
### Preparing data

Get data from pynapple to NeMoS-ready format:

- predictors and spikes must have same number of time points

In [ ]:
# enter code here

- predictors must be 2d, spikes 1d

In [ ]:
# enter code here

(current-inj-glm-users)=
### Fitting the model

- GLM objects need regularizers and observation models

In [ ]:
# enter code here

- call fit and retrieve parameters

In [ ]:
# enter code here

- generate model predictions.

In [ ]:
# enter code here

- and visualize!

In [ ]:
# and plot!
fig = doc_plots.current_injection_plot(current, spikes, firing_rate,
                                 # plot the predicted firing rate that has
                                 # been smoothed the same way as the
                                 # smoothed spike train
                                 predicted_firing_rates=smooth_predicted_fr)

- what do we see?

In [ ]:
# enter code here

- examine tuning curve &mdash; what do we see?

In [ ]:
# enter code here

(current-inj-basis-users)=
### Extending the model to use injection history

  - choose a length of time over which the neuron integrates the input current

In [ ]:
# enter code here

In [ ]:
doc_plots.plot_basis();

  - define a basis object

In [ ]:
# enter code here

  - create the design matrix
  - examine the features it contains

In [ ]:
# enter code here

In [ ]:
# in this plot, we're normalizing the amplitudes to make the comparison easier --
# the amplitude of these features will be fit by the model, so their un-scaled
# amplitudes is not informative
workshop_utils.plot_current_history_features(binned_current, current_history, basis,
                                             current_history_duration_sec)

  - create and fit the GLM
  - examine the parameters

In [ ]:
# enter code here

In [ ]:
print(f"firing_rate(t) = exp({history_model.coef_} * current(t) + {history_model.intercept_})")

(visualize-filter-users)=
- Visualize the current history model's learned filter.
- This filter is convolved with the input current and used to predict the firing
  rate.

In [ ]:
workshop_utils.plot_basis_filter(basis, history_model)

  - compare the predicted firing rate to the data and the old model
  - what do we see?

In [ ]:
# all this code is the same as above
history_pred_fr = history_model.predict(current_history)
history_pred_fr = history_pred_fr / bin_size
smooth_history_pred_fr = history_pred_fr.dropna().smooth(.05, size_factor=20)
workshop_utils.current_injection_plot(current, spikes, firing_rate,
                                      # compare against the old firing rate
                                      smooth_history_pred_fr, smooth_predicted_fr)

  - examine the predicted average firing rate and tuning curve
  - what do we see?

In [ ]:
# enter code here

In [ ]:
# Visualize tuning curve
tuning_curve_history_model = nap.compute_tuning_curves(smooth_history_pred_fr, current, bins=15, feature_names=["current"])
fig = doc_plots.tuning_curve_plot(tuning_curve)
tuning_curve_history_model.plot(color="tomato", label="glm (current history)")
tuning_curve_model.plot(color="tomato", linestyle='--', label="glm (instantaneous current)")
fig.axes[0].legend()

  - use log-likelihood to compare models

In [ ]:
# enter code here

(current-inj-score-users)=
### Finishing up

  - what if you want to compare models across datasets?

In [ ]:
# enter code here

## Further Exercises 

- what else can we do?



### Data citation

The data used in this tutorial is from the **Allen Brain Map**, with the [following citation](https://knowledge.brain-map.org/data/1HEYEW7GMUKWIQW37BO/summary):

**Contributors:** Agata Budzillo, Bosiljka Tasic, Brian R. Lee, Fahimeh Baftizadeh, Gabe Murphy, Hongkui Zeng, Jim Berg, Nathan Gouwens, Rachel Dalley, Staci A. Sorensen, Tim Jarsky, Uygar Sümbül Zizhen Yao

**Dataset:** Allen Institute for Brain Science (2020). Allen Cell Types Database -- Mouse Patch-seq [dataset]. Available from brain-map.org/explore/classes/multimodal-characterization.

**Primary publication:** Gouwens, N.W., Sorensen, S.A., et al. (2020). Integrated morphoelectric and transcriptomic classification of cortical GABAergic cells. Cell, 183(4), 935-953.E19. https://doi.org/10.1016/j.cell.2020.09.057

**Patch-seq protocol:** Lee, B. R., Budzillo, A., et al. (2021). Scaled, high fidelity electrophysiological, morphological, and transcriptomic cell characterization. eLife, 2021;10:e65482. https://doi.org/10.7554/eLife.65482

**Mouse VISp L2/3 glutamatergic neurons:** Berg, J., Sorensen, S. A., Miller, J., Ting, J., et al. (2021) Human neocortical expansion involves glutamatergic neuron diversification. Nature, 598(7879):151-158. doi: 10.1038/s41586-021-03813-8